# A Vision Transformer without Attention

## imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_addons as tfa

from emgdecompy.decomposition import *
from emgdecompy.contrast import *
from emgdecompy.viz import *
from emgdecompy.preprocessing import *
from db1_preprocess_utils import *
from feature_extraction import *
from experiment_one_utils import *

from tensorflow.keras import layers
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np
from sklearn.model_selection import KFold
from scipy.signal import stft
from tensorflow.keras import layers


# Setting seed for reproducibiltiy
SEED = 42
keras.utils.set_random_seed(SEED)

## Hyperparameters

In [2]:
class Config(object):
    # DATA
    batch_size = 36 
    buffer_size = batch_size * 2
    num_classes = 34
    

    # ARCHITECTURE
    patch_size = 4
    projected_dim = 96
    num_shift_blocks_per_stages = [2, 4, 8, 2]
    epsilon = 1e-5
    stochastic_depth_rate = 0.2
    mlp_dropout_rate = 0.2
    num_div = 12
    shift_pixel = 1
    mlp_expand_ratio = 2

    # OPTIMIZER
    lr_start = 1e-5
    lr_max = 1e-3
    weight_decay = 1e-4

    # TRAINING
    epochs = 150


config = Config()

## Import data

In [3]:
path = 'D:\\AI-Workspace\\sEMGClassification\\GestureClassificationUsingCViT\\data\\DB1\\raw'

signal_type='raw'
input_type='raw'
n_channels=64
low_cut=20
high_cut=50
order=6 
window_length=50
overlap=50
fs=2048

whiten=False
center=True
extend=True
extend_size=1
normalize=False
mu=0

n_folds = 3

start_subject = 1
total_subject = 20
session=[1,2]
ratio = 0.2

noise_db = [5,10,15]
std = 1
type_of_experiment = 3

X_train, y_label, X_test, y_test = get_experiment_data(path, subjects=start_subject, sessions=session,
                                                       signal_type='raw', input_type=input_type, 
                                                       channels=n_channels, low_cut=20, high_cut=500,
                                                      order=6, window_size=window_length, overlap=overlap, fs=fs, 
                                                      extend=extend, center=center, 
                                                      extend_size=extend_size, whiten=whiten,
                                                      normalize=False, mu=0, ratio=ratio)

input_size = X_train.shape[0:]
print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_test)}")

####################################################################################
Loading subject 1
####################################################################################
Centred.
Extended.
Size of the input data is (17152, 128, 102, 1)
The input label shape is (17152,)
The total number of classes is 34
************************************************************************************
Loaded RAW input data
************************************************************************************
Training samples: 17152
Validation samples: 4288


#### The MLP block

In [4]:
class MLP(layers.Layer):
    """Get the MLP layer for each shift block.

    Args:
        mlp_expand_ratio (int): The ratio with which the first feature map is expanded.
        mlp_dropout_rate (float): The rate for dropout.
    """

    def __init__(self, mlp_expand_ratio, mlp_dropout_rate, **kwargs):
        super().__init__(**kwargs)
        self.mlp_expand_ratio = mlp_expand_ratio
        self.mlp_dropout_rate = mlp_dropout_rate

    def build(self, input_shape):
        input_channels = input_shape[-1]
        initial_filters = int(self.mlp_expand_ratio * input_channels)

        self.mlp = keras.Sequential(
            [
                layers.Dense(units=initial_filters, activation=tf.nn.gelu,),
                layers.Dropout(rate=self.mlp_dropout_rate),
                layers.Dense(units=input_channels),
                layers.Dropout(rate=self.mlp_dropout_rate),
            ]
        )

    def call(self, x):
        x = self.mlp(x)
        return x


#### The DropPath layer

In [5]:

class DropPath(layers.Layer):
    """Drop Path also known as the Stochastic Depth layer.

    Refernece:
        - https://keras.io/examples/vision/cct/#stochastic-depth-for-regularization
        - github.com:rwightman/pytorch-image-models
    """

    def __init__(self, drop_path_prob, **kwargs):
        super().__init__(**kwargs)
        self.drop_path_prob = drop_path_prob

    def call(self, x, training=False):
        if training:
            keep_prob = 1 - self.drop_path_prob
            shape = (tf.shape(x)[0],) + (1,) * (len(tf.shape(x)) - 1)
            random_tensor = keep_prob + tf.random.uniform(shape, 0, 1)
            random_tensor = tf.floor(random_tensor)
            return (x / keep_prob) * random_tensor
        return x


In [6]:
class ShiftViTBlock(layers.Layer):
    """A unit ShiftViT Block

    Args:
        shift_pixel (int): The number of pixels to shift. Default to 1.
        mlp_expand_ratio (int): The ratio with which MLP features are
            expanded. Default to 2.
        mlp_dropout_rate (float): The dropout rate used in MLP.
        num_div (int): The number of divisions of the feature map's channel.
            Totally, 4/num_div of channels will be shifted. Defaults to 12.
        epsilon (float): Epsilon constant.
        drop_path_prob (float): The drop probability for drop path.
    """

    def __init__(
        self,
        epsilon,
        drop_path_prob,
        mlp_dropout_rate,
        num_div=12,
        shift_pixel=1,
        mlp_expand_ratio=2,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.shift_pixel = shift_pixel
        self.mlp_expand_ratio = mlp_expand_ratio
        self.mlp_dropout_rate = mlp_dropout_rate
        self.num_div = num_div
        self.epsilon = epsilon
        self.drop_path_prob = drop_path_prob

    def build(self, input_shape):
        self.H = input_shape[1]
        self.W = input_shape[2]
        self.C = input_shape[3]
        self.layer_norm = layers.LayerNormalization(epsilon=self.epsilon)
        self.drop_path = (
            DropPath(drop_path_prob=self.drop_path_prob)
            if self.drop_path_prob > 0.0
            else layers.Activation("linear")
        )
        self.mlp = MLP(
            mlp_expand_ratio=self.mlp_expand_ratio,
            mlp_dropout_rate=self.mlp_dropout_rate,
        )

    def get_shift_pad(self, x, mode):
        """Shifts the channels according to the mode chosen."""
        if mode == "left":
            offset_height = 0
            offset_width = 0
            target_height = 0
            target_width = self.shift_pixel
        elif mode == "right":
            offset_height = 0
            offset_width = self.shift_pixel
            target_height = 0
            target_width = self.shift_pixel
        elif mode == "up":
            offset_height = 0
            offset_width = 0
            target_height = self.shift_pixel
            target_width = 0
        else:
            offset_height = self.shift_pixel
            offset_width = 0
            target_height = self.shift_pixel
            target_width = 0
        crop = tf.image.crop_to_bounding_box(
            x,
            offset_height=offset_height,
            offset_width=offset_width,
            target_height=self.H - target_height,
            target_width=self.W - target_width,
        )
        shift_pad = tf.image.pad_to_bounding_box(
            crop,
            offset_height=offset_height,
            offset_width=offset_width,
            target_height=self.H,
            target_width=self.W,
        )
        return shift_pad

    def call(self, x, training=False):
        # Split the feature maps
        x_splits = tf.split(x, num_or_size_splits=self.C // self.num_div, axis=-1)

        # Shift the feature maps
        x_splits[0] = self.get_shift_pad(x_splits[0], mode="left")
        x_splits[1] = self.get_shift_pad(x_splits[1], mode="right")
        x_splits[2] = self.get_shift_pad(x_splits[2], mode="up")
        x_splits[3] = self.get_shift_pad(x_splits[3], mode="down")

        # Concatenate the shifted and unshifted feature maps
        x = tf.concat(x_splits, axis=-1)

        # Add the residual connection
        shortcut = x
        x = shortcut + self.drop_path(self.mlp(self.layer_norm(x)), training=training)
        return x


#### The PatchMerging layer

In [7]:

class PatchMerging(layers.Layer):
    """The Patch Merging layer.

    Args:
        epsilon (float): The epsilon constant.
    """

    def __init__(self, epsilon, **kwargs):
        super().__init__(**kwargs)
        self.epsilon = epsilon

    def build(self, input_shape):
        filters = 2 * input_shape[-1]
        self.reduction = layers.Conv2D(
            filters=filters, kernel_size=2, strides=2, padding="same", use_bias=False
        )
        self.layer_norm = layers.LayerNormalization(epsilon=self.epsilon)

    def call(self, x):
        # Apply the patch merging algorithm on the feature maps
        x = self.layer_norm(x)
        x = self.reduction(x)
        return x


In [8]:
input_size

(17152, 128, 102, 1)

#### Stacked Shift Blocks

In [9]:
# Note: This layer will have a different depth of stacking
# for different stages on the model.
class StackedShiftBlocks(layers.Layer):
    """The layer containing stacked ShiftViTBlocks.

    Args:
        epsilon (float): The epsilon constant.
        mlp_dropout_rate (float): The dropout rate used in the MLP block.
        num_shift_blocks (int): The number of shift vit blocks for this stage.
        stochastic_depth_rate (float): The maximum drop path rate chosen.
        is_merge (boolean): A flag that determines the use of the Patch Merge
            layer after the shift vit blocks.
        num_div (int): The division of channels of the feature map. Defaults to 12.
        shift_pixel (int): The number of pixels to shift. Defaults to 1.
        mlp_expand_ratio (int): The ratio with which the initial dense layer of
            the MLP is expanded Defaults to 2.
    """

    def __init__(
        self,
        epsilon,
        mlp_dropout_rate,
        num_shift_blocks,
        stochastic_depth_rate,
        is_merge,
        num_div=12,
        shift_pixel=1,
        mlp_expand_ratio=2,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.epsilon = epsilon
        self.mlp_dropout_rate = mlp_dropout_rate
        self.num_shift_blocks = num_shift_blocks
        self.stochastic_depth_rate = stochastic_depth_rate
        self.is_merge = is_merge
        self.num_div = num_div
        self.shift_pixel = shift_pixel
        self.mlp_expand_ratio = mlp_expand_ratio

    def build(self, input_shapes):
        # Calculate stochastic depth probabilities.
        # Reference: https://keras.io/examples/vision/cct/#the-final-cct-model
        dpr = [
            x
            for x in np.linspace(
                start=0, stop=self.stochastic_depth_rate, num=self.num_shift_blocks
            )
        ]

        # Build the shift blocks as a list of ShiftViT Blocks
        self.shift_blocks = list()
        for num in range(self.num_shift_blocks):
            self.shift_blocks.append(
                ShiftViTBlock(
                    num_div=self.num_div,
                    epsilon=self.epsilon,
                    drop_path_prob=dpr[num],
                    mlp_dropout_rate=self.mlp_dropout_rate,
                    shift_pixel=self.shift_pixel,
                    mlp_expand_ratio=self.mlp_expand_ratio,
                )
            )
        if self.is_merge:
            self.patch_merge = PatchMerging(epsilon=self.epsilon)

    def call(self, x, training=False):
        for shift_block in self.shift_blocks:
            x = shift_block(x, training=training)
        if self.is_merge:
            x = self.patch_merge(x)
        return x


## The ShiftViT model

In [10]:
class ShiftViTModel(keras.Model):
    """The ShiftViT Model.

    Args:
        data_augmentation (keras.Model): A data augmentation model.
        projected_dim (int): The dimension to which the patches of the image are
            projected.
        patch_size (int): The patch size of the images.
        num_shift_blocks_per_stages (list[int]): A list of all the number of shit
            blocks per stage.
        epsilon (float): The epsilon constant.
        mlp_dropout_rate (float): The dropout rate used in the MLP block.
        stochastic_depth_rate (float): The maximum drop rate probability.
        num_div (int): The number of divisions of the channesl of the feature
            map. Defaults to 12.
        shift_pixel (int): The number of pixel to shift. Default to 1.
        mlp_expand_ratio (int): The ratio with which the initial mlp dense layer
            is expanded to. Defaults to 2.
    """

    def __init__(
        self,
        projected_dim,
        patch_size,
        num_shift_blocks_per_stages,
        epsilon,
        mlp_dropout_rate,
        stochastic_depth_rate,
        num_div=12,
        shift_pixel=1,
        mlp_expand_ratio=2,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.patch_projection = layers.Conv2D(
            filters=projected_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding="same",
        )
        self.stages = list()
        for index, num_shift_blocks in enumerate(num_shift_blocks_per_stages):
            if index == len(num_shift_blocks_per_stages) - 1:
                # This is the last stage, do not use the patch merge here.
                is_merge = False
            else:
                is_merge = True
            # Build the stages.
            self.stages.append(
                StackedShiftBlocks(
                    epsilon=epsilon,
                    mlp_dropout_rate=mlp_dropout_rate,
                    num_shift_blocks=num_shift_blocks,
                    stochastic_depth_rate=stochastic_depth_rate,
                    is_merge=is_merge,
                    num_div=num_div,
                    shift_pixel=shift_pixel,
                    mlp_expand_ratio=mlp_expand_ratio,
                )
            )
        self.global_avg_pool = layers.GlobalAveragePooling2D()

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "patch_projection": self.patch_projection,
                "stages": self.stages,
                "global_avg_pool": self.global_avg_pool,
            }
        )
        return config

    def _calculate_loss(self, data, training=False):
        (images, labels) = data

        # Create patches and project the pathces.
        projected_patches = self.patch_projection(images)

        # Pass through the stages
        x = projected_patches
        for stage in self.stages:
            x = stage(x, training=training)

        # Get the logits.
        logits = self.global_avg_pool(x)

        # Calculate the loss and return it.
        total_loss = self.compiled_loss(labels, logits)
        return total_loss, labels, logits

    def train_step(self, inputs):
        with tf.GradientTape() as tape:
            total_loss, labels, logits = self._calculate_loss(
                data=inputs, training=True
            )

        # Apply gradients.
        train_vars = [
            self.patch_projection.trainable_variables,
            self.global_avg_pool.trainable_variables,
        ]
        train_vars = train_vars + [stage.trainable_variables for stage in self.stages]

        # Optimize the gradients.
        grads = tape.gradient(total_loss, train_vars)
        trainable_variable_list = []
        for (grad, var) in zip(grads, train_vars):
            for g, v in zip(grad, var):
                trainable_variable_list.append((g, v))
        self.optimizer.apply_gradients(trainable_variable_list)

        # Update the metrics
        self.compiled_metrics.update_state(labels, logits)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        _, labels, logits = self._calculate_loss(data=data, training=False)

        # Update the metrics
        self.compiled_metrics.update_state(labels, logits)
        return {m.name: m.result() for m in self.metrics}


## Instantiate the model

In [11]:
model = ShiftViTModel(
    projected_dim=config.projected_dim,
    patch_size=config.patch_size,
    num_shift_blocks_per_stages=config.num_shift_blocks_per_stages,
    epsilon=config.epsilon,
    mlp_dropout_rate=config.mlp_dropout_rate,
    stochastic_depth_rate=config.stochastic_depth_rate,
    num_div=config.num_div,
    shift_pixel=config.shift_pixel,
    mlp_expand_ratio=config.mlp_expand_ratio,
)

## Learning rate schedule

In [12]:
# Some code is taken from:
# https://www.kaggle.com/ashusma/training-rfcx-tensorflow-tpu-effnet-b2.
class WarmUpCosine(keras.optimizers.schedules.LearningRateSchedule):
    """A LearningRateSchedule that uses a warmup cosine decay schedule."""

    def __init__(self, lr_start, lr_max, warmup_steps, total_steps):
        """
        Args:
            lr_start: The initial learning rate
            lr_max: The maximum learning rate to which lr should increase to in
                the warmup steps
            warmup_steps: The number of steps for which the model warms up
            total_steps: The total number of steps for the model training
        """
        super().__init__()
        self.lr_start = lr_start
        self.lr_max = lr_max
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps
        self.pi = tf.constant(np.pi)

    def __call__(self, step):
        # Check whether the total number of steps is larger than the warmup
        # steps. If not, then throw a value error.
        if self.total_steps < self.warmup_steps:
            raise ValueError(
                f"Total number of steps {self.total_steps} must be"
                + f"larger or equal to warmup steps {self.warmup_steps}."
            )

        # `cos_annealed_lr` is a graph that increases to 1 from the initial
        # step to the warmup step. After that this graph decays to -1 at the
        # final step mark.
        cos_annealed_lr = tf.cos(
            self.pi
            * (tf.cast(step, tf.float32) - self.warmup_steps)
            / tf.cast(self.total_steps - self.warmup_steps, tf.float32)
        )

        # Shift the mean of the `cos_annealed_lr` graph to 1. Now the grpah goes
        # from 0 to 2. Normalize the graph with 0.5 so that now it goes from 0
        # to 1. With the normalized graph we scale it with `lr_max` such that
        # it goes from 0 to `lr_max`
        learning_rate = 0.5 * self.lr_max * (1 + cos_annealed_lr)

        # Check whether warmup_steps is more than 0.
        if self.warmup_steps > 0:
            # Check whether lr_max is larger that lr_start. If not, throw a value
            # error.
            if self.lr_max < self.lr_start:
                raise ValueError(
                    f"lr_start {self.lr_start} must be smaller or"
                    + f"equal to lr_max {self.lr_max}."
                )

            # Calculate the slope with which the learning rate should increase
            # in the warumup schedule. The formula for slope is m = ((b-a)/steps)
            slope = (self.lr_max - self.lr_start) / self.warmup_steps

            # With the formula for a straight line (y = mx+c) build the warmup
            # schedule
            warmup_rate = slope * tf.cast(step, tf.float32) + self.lr_start

            # When the current step is lesser that warmup steps, get the line
            # graph. When the current step is greater than the warmup steps, get
            # the scaled cos graph.
            learning_rate = tf.where(
                step < self.warmup_steps, warmup_rate, learning_rate
            )

        # When the current step is more that the total steps, return 0 else return
        # the calculated graph.
        return tf.where(
            step > self.total_steps, 0.0, learning_rate, name="learning_rate"
        )

## Run Experiment

In [13]:
def run_experiment(input_size, n_folds, n_batches, n_epochs, start_subject, total_subject, session, path, 
                       input_type,n_channels, extend, extend_size, center, whiten, ratio, noise_db, std,
                       dims, patch_size, num_shift_blocks_per_stages,
                        epsilon, mlp_dropout_rate, stochastic_depth_rate, num_div, shift_pixel,
                        mlp_expand_ratio, type_of_experiment, lr_start, lr_max, weight_decay):
   
    
    if  type_of_experiment == 1:
        print('************************************************')
        print('Experiment One')
        print('************************************************')
    elif type_of_experiment == 2:
        print('************************************************')
        print('Experiment Two')
        print('************************************************')
    elif type_of_experiment == 3:
        print('************************************************')
        print('Experiment Three')
        print('************************************************')
    else:
        print('************************************************')
        print('Enter Valid Experiment')
        print('************************************************')
        
        
    result = pd.DataFrame({
    'Subject': [0],
    'Validation_accuracy': [0],
    'No_noise': [0],
    '5_dB': [0],
    '10_dB': [0],
    '15_dB': [0],
    'Fold_1': [0],
    'Fold_2': [0],
    'Fold_3': [0]
    })
    
            
    total_steps = int((int(input_size[0]) / n_batches) * n_epochs)

    # Calculate the number of steps for warmup.
    warmup_epoch_percentage = 0.15
    warmup_steps = int(total_steps * warmup_epoch_percentage)

    # Initialize the warmupcosine schedule.
    scheduled_lrs = WarmUpCosine(lr_start=lr_start, lr_max=lr_max, warmup_steps=warmup_steps, total_steps=total_steps)
    
    
    for s in range(start_subject, (total_subject+1)):
        
        X_train, y_train, X_test, y_test = get_experiment_data(path, subjects=s, sessions=session,
                                                       signal_type='raw', input_type=input_type, 
                                                       channels=n_channels, low_cut=20, high_cut=500,
                                                      order=6, window_size=50, overlap=50, fs=2048, 
                                                      extend=extend,  extend_size=extend_size, 
                                                      center=center, whiten=whiten,
                                                      normalize=False, mu=0, ratio=ratio)

    
        #X_train, y_train, X_test, y_test = spilt_data(data, label, ratio)
        if input_type == 'raw':
            print('Adding noise to RAW input test data')
            X_test = np.expand_dims(X_test, axis=3)
            X_test_1 = add_noise_all_channel(X_test, noise_db[0], std)
            X_test_2 = add_noise_all_channel(X_test, noise_db[1], std)
            X_test_3 = add_noise_all_channel(X_test, noise_db[2], std)
            
            print("Size of the input test data is {}".format(X_test_3.shape))
      
            
        elif input_type == 'tkeo':
            print('Adding noise to  TKEO input test data')
            X_test = tkeo_image(X_test)
            X_test = np.expand_dims(X_test, axis=3)
            
            X_test_1 = add_noise_all_channel(X_test, noise_db[0], std)
            X_test_2 = add_noise_all_channel(X_test, noise_db[1], std)
            X_test_3 = add_noise_all_channel(X_test, noise_db[2], std)
            
            
            print("Size of the input test data is {}".format(X_test_3.shape))

            
        elif input_type == 'stft':
            print('Adding noise to STFT input test data')
            
            
            X_test_1 = add_noise_all_channel(X_test, noise_db[0], std)
            X_test_1 = stft_image(X_test_1, samples=X_test_1.shape[2])
            
            X_test_2 = add_noise_all_channel(X_test, noise_db[1], std)
            X_test_2 = stft_image(X_test_2, samples=X_test_2.shape[2])
            
            X_test_3 = add_noise_all_channel(X_test, noise_db[2], std)
            X_test_3 = stft_image(X_test_3, samples=X_test_3.shape[2])
            
            X_test = stft_image(X_test, samples=X_test.shape[2])
            
            
            print("Size of the input test data is {}".format(X_test_3.shape))
            
        else:
            print('Use correct input type')
        
        
        
        # Get the optimizer.
        optimizer = tfa.optimizers.AdamW(learning_rate=scheduled_lrs, weight_decay=weight_decay)
        
        kfold = KFold(n_splits=n_folds, shuffle=False)
        accuracy_per_fold = []
        loss_per_fold = []
        fold_no = 1
        
        convergence_speed = []
        
        callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=10, mode="auto")
        
        for train, test in kfold.split(X_train, y_train):
        
            
            
            model = ShiftViTModel(
            projected_dim=dims,
            patch_size=patch_size,
            num_shift_blocks_per_stages=num_shift_blocks_per_stages,
            epsilon=epsilon,
            mlp_dropout_rate=mlp_dropout_rate,
            stochastic_depth_rate=stochastic_depth_rate,
            num_div=num_div,
            shift_pixel=shift_pixel,
            mlp_expand_ratio=mlp_expand_ratio,
            )
            
            
            
            
            model.compile(optimizer=optimizer,
                          loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                          metrics=[keras.metrics.SparseCategoricalAccuracy(name="accuracy")])
            
            print('---------------------------------------------------')
            print(f'Training for fold {fold_no} -------')
            
            history = model.fit(X_train[train], y_train[train], batch_size=n_batches, 
                                epochs= n_epochs, verbose=1, 
                                callbacks=callback)
            
            conv_speed = len(history.history['accuracy'])
            convergence_speed.append(conv_speed)
            
            scores = model.evaluate(X_train[test], y_train[test], verbose=0)
            print(f'Score for fold  {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
            accuracy_per_fold.append(scores[1] *100)
            loss_per_fold.append(scores[0])
                  
            fold_no = fold_no + 1
        
        print("Average Score per fold ")
    
        for i in range(0, len(accuracy_per_fold)):
            print('-----------------------------------------------')
            print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {accuracy_per_fold[i]}%')
        print('-----------------------------------------------')
        print('Average Metrics for all folds: ')
        print(f'> Accuracy: {np.mean(accuracy_per_fold)} (+- {np.std(accuracy_per_fold)})')
        print(f'> Loss: {np.mean(loss_per_fold)}')
        print('-----------------------------------------------')
        
        print('************************************************')
        print(f'For subject {s} without noise')
        scores_0 = model.evaluate(X_test, y_test, verbose=0)
        print('The loss is {} and accuracy is {}%'.format(scores_0[0], (scores_0[1]*100)))
        print(f'For subject {s} and noise 5 dB')
        scores_1 = model.evaluate(X_test_1, y_test, verbose=0)
        print('The loss is {} and accuracy is {}%'.format(scores_1[0], (scores_1[1]*100)))
        print(f'For subject {s} and noise 10 dB')
        scores_2 = model.evaluate(X_test_2, y_test, verbose=0)
        print('The loss is {} and accuracy is {}%'.format(scores_2[0], (scores_2[1]*100)))
        print(f'For subject {s} and noise 15 dB')  
        scores_3 = model.evaluate(X_test_3, y_test, verbose=0) 
        print('The loss is {} and accuracy is {}%'.format(scores_3[0], (scores_3[1]*100)))
        print('************************************************') 

        
        result.at[s-1, 'Subject'] =  s
        result.at[s-1, 'Validation_accuracy'] =  np.mean(accuracy_per_fold)       
        result.at[s-1, 'No_noise'] =  scores_0[1]*100             
        result.at[s-1, '5_dB'] =  scores_1[1]*100    
        result.at[s-1, '10_dB'] =  scores_2[1]*100
        result.at[s-1, '15_dB'] =  scores_3[1]*100   
        
        result.at[s-1, 'Fold_1'] =  convergence_speed[0]
        result.at[s-1, 'Fold_2'] =  convergence_speed[1]
        result.at[s-1, 'Fold_3'] =  convergence_speed[2] 
        
        save_path = 'without_attention_raw_all.csv'
        result.to_csv(save_path, index=False)

In [ ]:
run_experiment(input_size, n_folds,  config.batch_size, config.epochs, start_subject, total_subject, session, path, 
                       input_type,n_channels, extend, extend_size, center, whiten, ratio, noise_db, std,
                       config.projected_dim, config.patch_size, config.num_shift_blocks_per_stages,
                        config.epsilon, config.mlp_dropout_rate, config.stochastic_depth_rate, config.num_div, config.shift_pixel,
                        config.mlp_expand_ratio, type_of_experiment, config.lr_start, config.lr_max, config.weight_decay)

************************************************
Experiment Three
************************************************
####################################################################################
Loading subject 1
####################################################################################
Centred.
Extended.
Size of the input data is (17152, 128, 102, 1)
The input label shape is (17152,)
The total number of classes is 34
************************************************************************************
Loaded RAW input data
************************************************************************************
Adding noise to RAW input test data
Size of the input test data is (4288, 128, 102, 1)
---------------------------------------------------
Training for fold 1 -------
Epoch 1/150
318/318 [==============================] - 101s 175ms/step - loss: 4.2236 - accuracy: 0.0270
Epoch 2/150
318/318 [==============================] - 55s 174ms/step - loss: 3.6007 - accuracy: 0.

318/318 [==============================] - 44s 140ms/step - loss: 0.0587 - accuracy: 0.9822
Epoch 72/150
318/318 [==============================] - 45s 140ms/step - loss: 0.0321 - accuracy: 0.9906
Epoch 73/150
318/318 [==============================] - 45s 143ms/step - loss: 0.0192 - accuracy: 0.9957
Epoch 74/150
318/318 [==============================] - 45s 142ms/step - loss: 0.0269 - accuracy: 0.9929
Epoch 75/150
318/318 [==============================] - 47s 148ms/step - loss: 0.0306 - accuracy: 0.9911
Epoch 76/150
318/318 [==============================] - 45s 142ms/step - loss: 0.0419 - accuracy: 0.9881
Epoch 77/150
318/318 [==============================] - 45s 141ms/step - loss: 0.0346 - accuracy: 0.9898
Epoch 78/150
318/318 [==============================] - 45s 141ms/step - loss: 0.0271 - accuracy: 0.9932
Score for fold  1: loss of 0.030169272795319557; accuracy of 99.05561208724976%
---------------------------------------------------
Training for fold 2 -------
Epoch 1/150
3

318/318 [==============================] - 44s 139ms/step - loss: 0.0330 - accuracy: 0.9918
Epoch 70/150
318/318 [==============================] - 44s 139ms/step - loss: 0.0193 - accuracy: 0.9966
Epoch 71/150
318/318 [==============================] - 44s 139ms/step - loss: 0.0511 - accuracy: 0.9869
Epoch 72/150
318/318 [==============================] - 44s 139ms/step - loss: 0.0249 - accuracy: 0.9946
Epoch 73/150
318/318 [==============================] - 44s 138ms/step - loss: 0.0193 - accuracy: 0.9966
Epoch 74/150
318/318 [==============================] - 44s 139ms/step - loss: 0.0277 - accuracy: 0.9941
Score for fold  2: loss of 0.050383519381284714; accuracy of 98.7755835056305%
---------------------------------------------------
Training for fold 3 -------
Epoch 1/150
318/318 [==============================] - 62s 138ms/step - loss: 3.9831 - accuracy: 0.0349
Epoch 2/150
318/318 [==============================] - 45s 140ms/step - loss: 3.4882 - accuracy: 0.0563
Epoch 3/150
318/

320/320 [==============================] - 45s 139ms/step - loss: 1.9280 - accuracy: 0.4338
Epoch 6/150
320/320 [==============================] - 45s 139ms/step - loss: 1.5496 - accuracy: 0.5537
Epoch 7/150
320/320 [==============================] - 44s 138ms/step - loss: 1.2698 - accuracy: 0.6304
Epoch 8/150
320/320 [==============================] - 44s 138ms/step - loss: 1.0201 - accuracy: 0.7064
Epoch 9/150
320/320 [==============================] - 44s 139ms/step - loss: 0.8485 - accuracy: 0.7588
Epoch 10/150
320/320 [==============================] - 44s 138ms/step - loss: 0.7273 - accuracy: 0.7880
Epoch 11/150
320/320 [==============================] - 45s 139ms/step - loss: 0.6196 - accuracy: 0.8239
Epoch 12/150
320/320 [==============================] - 44s 138ms/step - loss: 0.5354 - accuracy: 0.8478
Epoch 13/150
320/320 [==============================] - 44s 137ms/step - loss: 0.5125 - accuracy: 0.8486
Epoch 14/150
320/320 [==============================] - 45s 139ms/step -

320/320 [==============================] - 45s 139ms/step - loss: 0.0670 - accuracy: 0.9806
Epoch 84/150
320/320 [==============================] - 45s 139ms/step - loss: 0.0273 - accuracy: 0.9945
Epoch 85/150
320/320 [==============================] - 45s 140ms/step - loss: 0.0385 - accuracy: 0.9897
Epoch 86/150
320/320 [==============================] - 45s 140ms/step - loss: 0.0484 - accuracy: 0.9873
Epoch 87/150
320/320 [==============================] - 44s 139ms/step - loss: 0.0437 - accuracy: 0.9889
Epoch 88/150
320/320 [==============================] - 45s 139ms/step - loss: 0.0637 - accuracy: 0.9812
Epoch 89/150
320/320 [==============================] - 45s 140ms/step - loss: 0.0410 - accuracy: 0.9890
Epoch 90/150
320/320 [==============================] - 45s 140ms/step - loss: 0.0379 - accuracy: 0.9900
Epoch 91/150
320/320 [==============================] - 45s 141ms/step - loss: 0.0371 - accuracy: 0.9905
Epoch 92/150
320/320 [==============================] - 45s 140ms/st

...

316/316 [==============================] - 45s 142ms/step - loss: 0.3433 - accuracy: 0.8975
Epoch 13/150
316/316 [==============================] - 46s 145ms/step - loss: 0.3193 - accuracy: 0.9024
Epoch 14/150
316/316 [==============================] - 45s 143ms/step - loss: 0.3077 - accuracy: 0.9032
Epoch 15/150
316/316 [==============================] - 45s 143ms/step - loss: 0.2690 - accuracy: 0.9161
Epoch 16/150
316/316 [==============================] - 46s 144ms/step - loss: 0.2656 - accuracy: 0.9199
Epoch 17/150
316/316 [==============================] - 45s 144ms/step - loss: 0.2135 - accuracy: 0.9378
Epoch 18/150
316/316 [==============================] - 45s 144ms/step - loss: 0.2389 - accuracy: 0.9293
Epoch 19/150
316/316 [==============================] - 45s 144ms/step - loss: 0.2361 - accuracy: 0.9301
Epoch 20/150
316/316 [==============================] - 45s 142ms/step - loss: 0.1792 - accuracy: 0.9441
Epoch 21/150
316/316 [==============================] - 45s 144ms/st

316/316 [==============================] - 44s 139ms/step - loss: 0.1834 - accuracy: 0.9442
Epoch 25/150
316/316 [==============================] - 44s 140ms/step - loss: 0.1633 - accuracy: 0.9514
Epoch 26/150
316/316 [==============================] - 44s 139ms/step - loss: 0.1378 - accuracy: 0.9594
Epoch 27/150
316/316 [==============================] - 44s 139ms/step - loss: 0.1327 - accuracy: 0.9593
Epoch 28/150
316/316 [==============================] - 44s 139ms/step - loss: 0.1233 - accuracy: 0.9625
Epoch 29/150
316/316 [==============================] - 44s 140ms/step - loss: 0.1212 - accuracy: 0.9636
Epoch 30/150
316/316 [==============================] - 44s 139ms/step - loss: 0.1200 - accuracy: 0.9628
Epoch 31/150
316/316 [==============================] - 44s 139ms/step - loss: 0.0990 - accuracy: 0.9721
Epoch 32/150
316/316 [==============================] - 44s 140ms/step - loss: 0.1151 - accuracy: 0.9656
Epoch 33/150
316/316 [==============================] - 44s 140ms/st

316/316 [==============================] - 45s 143ms/step - loss: 0.0484 - accuracy: 0.9907
Epoch 39/150
316/316 [==============================] - 46s 144ms/step - loss: 0.0557 - accuracy: 0.9874
Epoch 40/150
316/316 [==============================] - 45s 144ms/step - loss: 0.0560 - accuracy: 0.9885
Epoch 41/150
316/316 [==============================] - 45s 144ms/step - loss: 0.0501 - accuracy: 0.9907
Epoch 42/150
316/316 [==============================] - 45s 143ms/step - loss: 0.0570 - accuracy: 0.9881
Epoch 43/150
316/316 [==============================] - 45s 144ms/step - loss: 0.0468 - accuracy: 0.9904
Epoch 44/150
316/316 [==============================] - 45s 143ms/step - loss: 0.0546 - accuracy: 0.9882
Epoch 45/150
316/316 [==============================] - 46s 144ms/step - loss: 0.0523 - accuracy: 0.9898
Score for fold  3: loss of 0.08479998260736465; accuracy of 97.4440336227417%
Average Score per fold 
-----------------------------------------------
> Fold 1 - Loss: 0.0984

313/313 [==============================] - 44s 140ms/step - loss: 0.0486 - accuracy: 0.9860
Epoch 54/150
313/313 [==============================] - 44s 140ms/step - loss: 0.0524 - accuracy: 0.9849
Epoch 55/150
313/313 [==============================] - 44s 140ms/step - loss: 0.0428 - accuracy: 0.9875
Epoch 56/150
313/313 [==============================] - 44s 139ms/step - loss: 0.0533 - accuracy: 0.9842
Epoch 57/150
313/313 [==============================] - 43s 139ms/step - loss: 0.0592 - accuracy: 0.9822
Epoch 58/150
313/313 [==============================] - 44s 140ms/step - loss: 0.0603 - accuracy: 0.9816
Epoch 59/150
313/313 [==============================] - 43s 139ms/step - loss: 0.0419 - accuracy: 0.9886
Epoch 60/150
313/313 [==============================] - 43s 139ms/step - loss: 0.0394 - accuracy: 0.9902
Epoch 61/150
313/313 [==============================] - 44s 139ms/step - loss: 0.0383 - accuracy: 0.9895
Epoch 62/150
313/313 [==============================] - 44s 139ms/st

313/313 [==============================] - 45s 144ms/step - loss: 0.1014 - accuracy: 0.9738
Epoch 27/150
313/313 [==============================] - 45s 143ms/step - loss: 0.0981 - accuracy: 0.9737
Epoch 28/150
313/313 [==============================] - 45s 143ms/step - loss: 0.0899 - accuracy: 0.9766
Epoch 29/150
313/313 [==============================] - 45s 144ms/step - loss: 0.0991 - accuracy: 0.9743
Epoch 30/150
313/313 [==============================] - 45s 143ms/step - loss: 0.0693 - accuracy: 0.9830
Epoch 31/150
313/313 [==============================] - 45s 144ms/step - loss: 0.0798 - accuracy: 0.9786
Epoch 32/150
313/313 [==============================] - 45s 145ms/step - loss: 0.0892 - accuracy: 0.9773
Epoch 33/150
313/313 [==============================] - 45s 144ms/step - loss: 0.0663 - accuracy: 0.9839
Epoch 34/150
313/313 [==============================] - 45s 144ms/step - loss: 0.0662 - accuracy: 0.9842
Epoch 35/150
313/313 [==============================] - 45s 143ms/st